In [ ]:
# Define the storage account details
storage_account_name = ""
storage_account_key = ""

# Set the configuration for the storage account
configs = {
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net": storage_account_key
}

# Define the mount point
mount_point = "/mnt/tokyoolympic"

# Unmount if the mount point already exists to avoid conflicts
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mount_point)

# Mount the ADLS storage
try:
    dbutils.fs.mount(
        source=f"abfss://tokyo-olympics-data@{storage_account_name}.dfs.core.windows.net/",
        mount_point=mount_point,
        extra_configs=configs
    )
    print(f"Mount successful: {mount_point}")
except Exception as e:
    print(f"Error mounting storage: {e}")

# Verify the mount by listing the contents
try:
    display(dbutils.fs.ls(mount_point))
except Exception as e:
    print(f"Error listing contents of mount point: {e}")


Error mounting storage: Unsupported Azure Scheme: abfss
Error listing contents of mount point: An error occurred while calling o1923.ls.
: java.io.FileNotFoundException: Operation failed: "The specified path does not exist.", 404, GET, https://dbstoragekt7zpnrndjfbm.dfs.core.windows.net/root?upn=false&resource=filesystem&maxResults=5000&directory=3109534669391467/mnt/tokyoolympic&timeout=90&recursive=false&sig=XXXXX&st=2024-07-02T17:03:51Z&se=2024-07-03T13:03:51Z&sv=2019-02-02&spr=https&sp=racwdl&sr=c, PathNotFound, "The specified path does not exist. RequestId:f55de7aa-b01f-0099-06ab-cc9ed0000000 Time:2024-07-02T18:16:29.2592581Z"
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.checkException(AzureBlobFileSystem.java:1563)
	at shaded.databricks.azurebfs.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.listStatus(AzureBlobFileSystem.java:625)
	at com.databricks.common.filesystem.LokiFileSystem.listStatus(LokiFileSystem.scala:132)
	at com.databricks.bac

In [ ]:
# List Contents of the Container

# Define the storage account details
storage_account_name = ""
storage_account_key = ""

# Configure the Spark session
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

# Define the container path
container_path = f"abfss://tokyo-olympics-data@{storage_account_name}.dfs.core.windows.net/"

# List the contents of the container
try:
    files = dbutils.fs.ls(container_path)
    for file in files:
        print(file.path)
except Exception as e:
    print(f"Error listing contents of container: {e}")


abfss://tokyo-olympics-data@tokyoolymicsdataali.dfs.core.windows.net/raw-data/
abfss://tokyo-olympics-data@tokyoolymicsdataali.dfs.core.windows.net/transformed-data/


In [ ]:
# Define the storage account details
storage_account_name = ""
storage_account_key = ""

# Configure the Spark session
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

# Path to the data directory in ADLS
data_directory_path = f"abfss://tokyo-olympics-data@{storage_account_name}.dfs.core.windows.net/raw-data/"

# List all files in the directory
try:
    files = dbutils.fs.ls(data_directory_path)
    csv_files = [file.path for file in files if file.path.endswith(".csv")]
except Exception as e:
    print(f"Error listing contents of directory: {e}")

# Load each CSV file into a separate DataFrame and assign to variables dynamically
for csv_file in csv_files:
    try:
        # Extract file name without path and extension to use as variable name
        file_name = csv_file.split("/")[-1].split(".csv")[0]
        
        # Load CSV file into DataFrame
        df = spark.read.csv(csv_file, header=True, inferSchema=True)
        print(f"Loaded data from: {csv_file}")
        
        # Assign DataFrame to a variable named after the file name
        globals()[file_name] = df  # Use locals() if you want to store in local scope
        
        # Example: Show first few rows of each DataFrame
        print(f"First few rows of DataFrame {file_name}:")
        globals()[file_name].show()
        
        # Example: Print schema of each DataFrame
        print(f"Schema of DataFrame {file_name}:")
        globals()[file_name].printSchema()
        
    except Exception as e:
        print(f"Error reading data from {csv_file}: {e}")

# Now you can access each DataFrame by its file name variable, e.g., athlete, sport, event, etc.
# Example:
# athlete.show()
# sport.printSchema()
# event.show()


Loaded data from: abfss://tokyo-olympics-data@tokyoolymicsdataali.dfs.core.windows.net/raw-data/athletes.csv
First few rows of DataFrame athletes:
+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois

In [ ]:
athletes.show()
athletes.printSchema()


+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
entriesgender.show()
entriesgender.printSchema()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [ ]:
from pyspark.sql.types import IntegerType

# Manipulation of the table 'entriesgender'

In [ ]:
entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType())).withColumn("Male",col("Male").cast(IntegerType())).withColumn("Female",col("Female").cast(IntegerType()))

entriesgender.show()
entriesgender.printSchema()

In [ ]:
# change the type of columns to IntegerType and add % columns

entriesgender = entriesgender.withColumn("% of Female",col("Female")/col("Total")*100)\
.withColumn("% of Male",col("Male")/col("Total")*100)\
.withColumn("Total",col("Total").cast(IntegerType()))

entriesgender.show()
entriesgender.printSchema()

+--------------------+------+----+-----+------------------+------------------+
|          Discipline|Female|Male|Total|       % of Female|         % of Male|
+--------------------+------+----+-----+------------------+------------------+
|      3x3 Basketball|    32|  32|   64|              50.0|              50.0|
|             Archery|    64|  64|  128|              50.0|              50.0|
| Artistic Gymnastics|    98|  98|  196|              50.0|              50.0|
|   Artistic Swimming|   105|   0|  105|             100.0|               0.0|
|           Athletics|   969|1072| 2041| 47.47672709456149| 52.52327290543851|
|           Badminton|    86|  87|  173| 49.71098265895954| 50.28901734104046|
|   Baseball/Softball|    90| 144|  234| 38.46153846153847| 61.53846153846154|
|          Basketball|   144| 144|  288|              50.0|              50.0|
|    Beach Volleyball|    48|  48|   96|              50.0|              50.0|
|              Boxing|   102| 187|  289|35.294117647

# Creation of 'sports' table from the table 'athletes'

In [ ]:
sports = athletes.groupBy('Discipline').count()
sports.show()

+--------------------+-----+
|          Discipline|count|
+--------------------+-----+
|              Tennis|  178|
|              Boxing|  270|
|   Marathon Swimming|   49|
|                Golf|  115|
|              Rowing|  496|
|   Baseball/Softball|  220|
|                Judo|  373|
|             Sailing|  336|
|            Swimming|  743|
|Cycling BMX Frees...|   19|
|          Basketball|  280|
|            Handball|  343|
| Rhythmic Gymnastics|   95|
|              Karate|   77|
|           Triathlon|  106|
|           Badminton|  164|
|        Canoe Sprint|  236|
|           Athletics| 2068|
|       Cycling Track|  208|
|    Beach Volleyball|   90|
+--------------------+-----+
only showing top 20 rows



# Joined 'athletes-coaches' table

In [ ]:
# Join on multiple columns
athletes_coaches = athletes.join(coaches, (athletes.Discipline == coaches.Discipline) & (athletes.Country == coaches.Country), "inner")

# Select columns to keep and remove duplicates
athletes_coaches = athletes_coaches.select(
    athletes["PersonName"].alias("AthleteName"),
    athletes["Discipline"],
    athletes["Country"],
    coaches["Name"].alias("CoachName")
)

# Show the result
athletes_coaches.show()


+--------------------+-----------------+--------------------+--------------------+
|         AthleteName|       Discipline|             Country|           CoachName|
+--------------------+-----------------+--------------------+--------------------+
|      ABALDE Alberto|       Basketball|               Spain|ZAMORA PEDREIRA J...|
|       ABALDE Tamara|       Basketball|               Spain|ZAMORA PEDREIRA J...|
|           ABALO Luc|         Handball|              France|   KRUMBHOLZ Olivier|
|        ABBINGH Lois|         Handball|         Netherlands|  MAYONNADE Emmanuel|
|       ABBOTT Monica|Baseball/Softball|United States of ...|     WEINSTEIN Jerry|
|      ABDALLA Maryam|Artistic Swimming|               Egypt|         MOHAMED Mai|
|      ABDALLAH Shahd|Artistic Swimming|               Egypt|         MOHAMED Mai|
|  ABDELSALAM Mohamed|         Football|               Egypt|       SIAM Zaghloul|
|     ABDULHAMID Saud|         Football|        Saudi Arabia|      MOHAMED Sayeda|
|  A

In [ ]:
# athletes_coaches = athletes.join(coaches, (athletes.Discipline == coaches.Discipline) & (athletes.Country == coaches.Country), "inner")

# athletes_coaches = athletes_coaches.withColumn("AthleteName",athletes_coaches["personName"])\
#     .withColumn("CoachName",athletes_coaches["Name"])

# athletes_coaches = athletes_coaches.select(athletes_coaches["Country"], "AthleteName", "CoachName","Discipline")

# athletes_coaches.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-2443687673721993>", line 6, in <module>
    athletes_coaches = athletes_coaches.select(athletes_coaches["Country"], "AthleteName", "CoachName","Discipline")
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 48, in wrapper
    res = func(*args, **kwargs)
  File "/databricks/spark/python/pyspark/sql/dataframe.py", line 2918, in __getitem__
    jc = self._jdf.apply(item)
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/databricks/spark/python/pyspark/errors/exceptions.py", line 234, in deco
    raise converted from None
pyspark.errors.exceptions.AnalysisException: [AMBIGUOUS_REFERENCE] Reference `Country` is ambiguous, could be: [`Country`, `Country`]

# Save all changements

In [ ]:

entriesgender.repartition(1).write.mode("overwrite").option("header",'true').csv("abfss://tokyo-olympics-data@{storage account name}.dfs.core.windows.net/transformed-data/entriesgender")
sports.repartition(1).write.mode("overwrite").option("header",'true').csv("abfss://tokyo-olympics-data@{storage account name}.dfs.core.windows.net/transformed-data//sports")
athletes_coaches.repartition(1).write.mode("overwrite").option("header",'true').csv("abfss://tokyo-olympics-data@{storage account name}.dfs.core.windows.net/transformed-data/athletes_coaches")

